In [ ]:
! pip install librosa
import pandas as pd
import os
import librosa
import IPython.display as ipd
import matplotlib.pyplot as plt
import numpy as np
import sklearn
import tensorflow as tf
import scipy
import gc

from scipy.io import wavfile
from scipy.stats import skew
from tqdm import tqdm, tqdm_pandas

tqdm.pandas()

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import GridSearchCV

from warnings import simplefilter

In [ ]:
simplefilter(action='ignore', category=FutureWarning)

# Loading data

In [ ]:
train = pd.read_csv("../input/freesound-audio-tagging/train.csv")
test = pd.read_csv("../input/freesound-audio-tagging/sample_submission.csv")

train_path = '../input/freesound-audio-tagging/audio_train/'
test_path = '../input/freesound-audio-tagging/audio_test/'

In [ ]:
train

In [ ]:
train.info()

In [ ]:
test

In [ ]:
train = train[:7000]

# Data preprocessing

In [ ]:
def audio_magic(fileName, path):
    data, _ = librosa.core.load(path + fileName, sr = 44100)
    try:
        mfcc = librosa.feature.mfcc(data, sr = 44100, n_mfcc=20)
        rms = librosa.feature.rms(data)[0]
        chroma_stft = librosa.feature.chroma_stft(data)[0]
        zcr = librosa.feature.zero_crossing_rate(data)[0]
        rolloff = librosa.feature.spectral_rolloff(data)[0]
        spectral_centroid = librosa.feature.spectral_centroid(data)[0]
        spectral_bandwidth = librosa.feature.spectral_bandwidth(data)[0]
        mfcc_trunc = np.hstack((np.mean(mfcc, axis=1), np.std(mfcc, axis=1), skew(mfcc, axis = 1), np.max(mfcc, axis = 1), np.median(mfcc, axis = 1), np.min(mfcc, axis = 1)))
        return pd.Series(np.hstack((mfcc_trunc, rms, chroma_stft, zcr, rolloff, spectral_centroid, spectral_bandwidth)))
    except:
        print('Error with audio')
        return pd.Series([0]*20)

In [ ]:
train_df = pd.DataFrame()
train_df['fname'] = train['fname']
train_df = train_df['fname'].progress_apply(audio_magic, path=train_path)

In [ ]:
train_df.head()

In [ ]:
train_df = train_df.iloc[:, 0:20]

In [ ]:
train_df['fname'] = train['fname']
train_df['label'] = train['label']
train_df.head()

In [ ]:
X_data = train_df.drop(['fname', 'label'], axis=1)

In [ ]:
genre_list = train_df.iloc[:, -1]
encoder = LabelEncoder()
y = encoder.fit_transform(genre_list)

In [ ]:
scaler = StandardScaler()
X = scaler.fit_transform(np.array(X_data.iloc[:, :-1], dtype = float))

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2)

# Model building

In [ ]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(41, activation='softmax')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(
    X_train,
    y_train,
    epochs=50,
    batch_size=256,
    validation_data=(X_val, y_val)
)

# Test data

In [ ]:
test_df = pd.DataFrame()
test_df['fname'] = test['fname']
test_df = test_df['fname'].progress_apply(audio_magic, path=test_path)

In [ ]:
test_df = test_df.iloc[:, 0:20]

In [ ]:
test_df['fname'] = test['fname']
test_df['label'] = np.zeros((len(test)))

In [ ]:
test_df.head()

In [ ]:
test_df.info()

In [ ]:
X_test_data = test_df.drop(['fname','label'],axis=1)

In [ ]:
X_test_s = scaler.fit_transform(np.array(X_test_data.iloc[:, :-1], dtype = float))

# Predictions

In [ ]:
preds = model.predict(X_test_s)

In [ ]:
model.save('sound.h5')

In [ ]:
score = np.max(preds,axis=1)
score = score.tolist()

In [ ]:
output = np.argmax(preds, axis=1)

In [ ]:
output = np.argmax(preds, axis=1)
submission = pd.DataFrame()
submission['fname'] = test['fname']
submission['label'] = encoder.inverse_transform(output)
submission.to_csv('./submission.csv', index=False)

In [ ]:
submission